## Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
from src.modules.feature_engineering import scale_encoder, num_null_replacement
from src.modules.load_source_data import load_data_set, load_agg_data

In [2]:
# get X and y (target variable)
# Load data
X, y = load_data_set()

# Save y
y.to_csv('./data/target.csv', index=False)
print(X.shape)
print(y.shape)

(9625, 27)
(9625,)


In [3]:
pd.set_option("display.max_columns", None)
X.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,year,month,day,day_of_week,week_of_year,crs_dep_hour,crs_arr_hour
0,2018-10-30,UA,UA_CODESHARE,UA,3997,EV,N13903,3997,12266,IAH,"Houston, TX",13256,MFE,"Mission/McAllen/Edinburg, TX",1455,1617,N,82,1,316,2018,10,30,2,44,14,16
1,2018-10-30,F9,F9,F9,529,F9,N301FR,529,10599,BHM,"Birmingham, AL",11292,DEN,"Denver, CO",1315,1518,N,183,1,1083,2018,10,30,2,44,13,15
2,2018-10-30,AA,AA,AA,2318,AA,N167AN,2318,11298,DFW,"Dallas/Fort Worth, TX",12892,LAX,"Los Angeles, CA",1740,1853,N,193,1,1235,2018,10,30,2,44,17,18
3,2018-10-30,AA,AA,AA,2375,AA,N962NN,2375,11298,DFW,"Dallas/Fort Worth, TX",14869,SLC,"Salt Lake City, UT",1855,2048,N,173,1,989,2018,10,30,2,44,18,20
4,2018-10-31,UA,UA,UA,746,UA,N68836,746,12173,HNL,"Honolulu, HI",14771,SFO,"San Francisco, CA",1433,2236,N,303,1,2398,2018,10,31,3,44,14,22


In [4]:
# drop features
train = X.copy()
to_drop = ['mkt_carrier', 'tail_num', 'origin_city_name', 'dest_city_name', 'dup', 'fl_date', 'branded_code_share', 'op_unique_carrier', 'origin', 'dest', 'flights']
train = train.drop(to_drop, axis=1)

In [5]:
train.shape

(9625, 16)

In [6]:
#load features - Passengers, fuel_comsumption
train = load_agg_data(train)

In [7]:
train.shape

(9625, 52)

In [8]:
train.head()

,mkt_unique_carrier,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week,week_of_year,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_air_time,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_air_time,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_air_time,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_seats,month_flight_passengers,month_flight_avg_seats,month_flight_avg_passengers,carrier_month_seats,carrier_month_passengers,carrier_month_avg_seats,carrier_month_avg_passengers,month_avg_fuel_comsumption,orig_airport_month_flight_seats,orig_airport_month_passengers,dest_airport_month_flight_seats,dest_airport_month_passengers
0,UA,3997,3997,12266,13256,1455,1617,82,316,2018,10,30,2,44,14,16,9.749340,53.232190,36.952381,4.238095,13.892857,0.0,10.690476,8.732231,53.522314,18.540323,12.298387,10.580645,0.0,28.370968,6.329897,53.782032,23.574468,7.212766,8.751773,0.0,23.503546,60613,1764,46933.0,27677.0,150.426277,82.035018,55003329,45704398,169.542505,134.985112,1.378381e+08,21173178,17176426,405577,288439
1,F9,529,529,10599,11292,1315,1518,183,1083,2018,10,30,2,44,13,15,9.230769,154.653846,9.200000,0.000000,14.500000,0.0,13.600000,6.806452,152.129032,8.300000,1.400000,18.100000,0.0,13.200000,10.578947,153.894737,12.625000,0.000000,10.000000,0.0,15.625000,7183,97485,4728.0,4203.0,176.600000,159.452381,9175856,7825867,182.916216,150.583777,1.878024e+07,1624146,1304863,30526301,26412747
2,AA,2318,2318,11298,12892,1740,1853,193,1235,2018,10,30,2,44,17,18,12.060780,167.094037,21.235808,11.449782,8.200873,0.0,23.078603,2.587422,167.238424,22.679245,4.456604,9.173585,0.0,23.649057,13.709353,168.212950,25.615000,5.910000,8.015000,0.0,22.035000,100056,83560,414528.0,376739.0,215.590837,192.751409,74614837,62505008,168.496511,136.733012,1.819297e+08,33551297,27909683,41526408,34347751
3,AA,2375,2375,11298,14869,1855,2048,173,989,2018,10,30,2,44,18,20,15.659942,138.567723,14.092784,8.505155,7.144330,0.0,34.474227,9.633648,139.055031,25.697368,4.361842,5.651316,0.0,20.118421,22.419118,138.664216,16.912281,6.333333,6.853801,0.0,25.140351,100056,40380,128478.0,107398.0,157.619886,128.523313,74614837,62505008,168.496511,136.733012,1.819297e+08,33551297,27909683,11730371,9839898
4,UA,746,746,12173,14771,1433,2236,303,2398,2018,10,31,3,44,14,22,0.732704,274.103774,13.916667,0.416667,11.416667,0.0,26.527778,3.350202,269.595142,22.764706,0.000000,7.558824,0.0,40.500000,3.611321,274.694340,9.094340,0.000000,10.377358,0.0,28.132075,18549,59900,195967.0,160718.0,241.446000,194.466981,55003329,45704398,169.542505,134.985112,1.378381e+08,9959578,8131626,28213693,22922751


In [10]:
sorted(list(train.columns))

['arr_hour_avg_air_time',
 'arr_hour_avg_arr_delay',
 'arr_hour_avg_carrier_delay',
 'arr_hour_avg_late_aircraft_delay',
 'arr_hour_avg_nas_delay',
 'arr_hour_avg_security_delay',
 'arr_hour_avg_weather_delay',
 'carrier_month_avg_passengers',
 'carrier_month_avg_seats',
 'carrier_month_passengers',
 'carrier_month_seats',
 'crs_arr_hour',
 'crs_arr_time',
 'crs_dep_hour',
 'crs_dep_time',
 'crs_elapsed_time',
 'day',
 'day_of_week',
 'day_of_week_avg_air_time',
 'day_of_week_avg_arr_delay',
 'day_of_week_avg_carrier_delay',
 'day_of_week_avg_late_aircraft_delay',
 'day_of_week_avg_nas_delay',
 'day_of_week_avg_security_delay',
 'day_of_week_avg_weather_delay',
 'dest_airport_id',
 'dest_airport_month_flight_seats',
 'dest_airport_month_passengers',
 'dest_total_flights',
 'distance',
 'mkt_carrier_fl_num',
 'mkt_unique_carrier',
 'month',
 'month_avg_air_time',
 'month_avg_arr_delay',
 'month_avg_carrier_delay',
 'month_avg_fuel_comsumption',
 'month_avg_late_aircraft_delay',
 'month_

In [11]:
# Remove nulls for numeric values
train_n_null = num_null_replacement(train)

Missing data:
-------------
                                     Total   Percent
month_flight_avg_passengers           1849  0.192104
month_flight_seats                    1849  0.192104
month_flight_passengers               1849  0.192104
month_flight_avg_seats                1849  0.192104
arr_hour_avg_weather_delay              29  0.003013
arr_hour_avg_security_delay             29  0.003013
arr_hour_avg_late_aircraft_delay        29  0.003013
arr_hour_avg_nas_delay                  29  0.003013
arr_hour_avg_carrier_delay              29  0.003013
month_avg_late_aircraft_delay           22  0.002286
month_avg_security_delay                22  0.002286
month_avg_nas_delay                     22  0.002286
month_avg_weather_delay                 22  0.002286
month_avg_carrier_delay                 22  0.002286
day_of_week_avg_nas_delay                5  0.000519
day_of_week_avg_weather_delay            5  0.000519
day_of_week_avg_security_delay           5  0.000519
day_of_week_avg_la

In [12]:
train_n_null.dtypes[train_n_null.dtypes == 'object']

mkt_unique_carrier    object
dtype: object

In [15]:
train_cleaned, scaler = scale_encoder(train_n_null)

In [16]:
train_cleaned.head()

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week,week_of_year,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_air_time,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_air_time,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_air_time,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_seats,month_flight_passengers,month_flight_avg_seats,month_flight_avg_passengers,carrier_month_seats,carrier_month_passengers,carrier_month_avg_seats,carrier_month_avg_passengers,month_avg_fuel_comsumption,orig_airport_month_flight_seats,orig_airport_month_passengers,dest_airport_month_flight_seats,dest_airport_month_passengers,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_VX,mkt_unique_carrier_WN
0,0.698804,0.698276,-0.258539,0.381238,0.274783,0.261833,-0.771809,-0.762133,-1.018139,1.005961,1.634739,-0.489136,1.167812,0.218584,0.285853,0.497332,-0.780933,0.975995,0.082092,-0.065738,-0.102704,-1.023031,0.453900,-0.777971,-0.162381,1.522247,-0.400371,-0.183872,0.131708,0.096928,-0.774255,0.047914,0.366677,-0.530028,-0.133627,-0.039800,0.320935,-1.195352,-0.312537,-0.456712,0.406251,-0.304022,-0.254159,-0.259971,0.632586,0.421258,-0.085478,0.295418,0.252337,-1.188139,-1.169661,0,0,0,0,0,0,0,0,1,0,0
1,-1.159166,-1.158730,-1.348294,-0.911835,-0.011907,0.067798,0.630635,0.534271,-1.018139,1.005961,1.634739,-0.489136,1.167812,0.013148,0.089777,0.439307,0.679910,-0.694443,-0.434385,-0.018364,-0.102704,-0.832426,0.191281,0.643978,-0.921878,-0.416692,0.260800,-0.183872,-1.076003,0.520879,0.668109,-0.530107,-0.437365,-0.426403,-0.133627,-0.510606,-1.046826,1.261097,-0.777097,-0.762170,0.813224,1.105522,-2.223217,-2.195894,1.610032,1.641769,-2.450965,-1.089429,-1.071751,0.950548,1.016383,0,0,0,0,1,0,0,0,0,0,0
2,-0.200715,-0.200776,-0.891342,0.141585,0.858403,0.724379,0.769490,0.791186,-1.018139,1.005961,1.634739,-0.489136,1.167812,0.834893,0.678004,0.755968,0.859094,0.030002,0.960949,-0.509875,-0.102704,-0.211475,-0.384069,0.861861,0.144592,0.127110,-0.524093,-0.183872,-0.244188,0.833216,0.874397,0.155633,0.221451,-0.591193,-0.133627,-0.127557,1.330641,0.903745,3.733669,4.085497,1.419489,1.711800,0.588484,0.598686,0.556137,0.558022,0.790552,1.172280,1.147761,1.731597,1.680372,1,0,0,0,0,0,0,0,0,0,0
3,-0.170178,-0.170254,-0.891342,1.443216,1.093898,1.106567,0.491779,0.375390,-1.018139,1.005961,1.634739,-0.489136,1.167812,1.040329,1.070156,1.158692,0.448211,-0.399943,0.602100,-0.592316,-0.102704,0.535061,0.576826,0.455446,0.368438,0.110251,-0.833804,-0.183872,-0.525250,1.702237,0.448677,-0.303782,0.268642,-0.687593,-0.133627,0.058012,1.330641,-0.204365,0.585048,0.580667,0.518104,0.542394,0.588484,0.598686,0.556137,0.558022,0.790552,1.172280,1.147761,-0.384035,-0.370409,1,0,0,0,0,0,0,0,0,0,0
4,-1.042909,-1.042533,-0.319335,1.378694,0.229732,1.475036,2.296904,2.756921,-1.018139,1.005961,1.749259,0.022763,1.167812,0.218584,1.462307,-0.511575,2.400425,-0.410543,-0.383608,-0.258952,-0.102704,0.014483,-0.280048,2.337886,0.150930,-0.665767,-0.666078,-0.183872,1.097260,-0.174319,2.408518,-0.716490,-0.437365,-0.395075,-0.133627,0.236792,-0.755866,0.296568,1.327916,1.274499,1.821508,1.743035,-0.254159,-0.259971,0.632586,0.421258,-0.085478,-0.498949,-0.502227,0.786344,0.724345,0,0,0,0,0,0,0,0,1,0,0


In [17]:
train_cleaned.to_csv('./data/train.csv')